1. Data Description

In [ ]:
library(tidyverse)
library(tidymodels)

In [ ]:
players <- read_csv("players.csv")
sessions <- read_csv("sessions.csv")

In [ ]:
combined_data <- inner_join(sessions, players)
combined_data

In [ ]:
average_hr_played <- combined_data |>
  group_by(average_hr_played = round(mean(played_hours, na.rm = TRUE), 2))
average_hr_played

- 1535 observations in total.
- 11 variables in total , including 7 variables in players and 5 variables in sessions.
-  players.csv variables
	•	experience — categorical; self-reported experience level
	•	subscribe — logical; whether subscribed to newsletter
	•	hashedEmail — character ID; used for merging
	•	played_hours — numeric; self-reported past hours played
	•	name — character; player name, not used for modeling
	•	gender — categorical; gender identity
	•	Age — numeric; age of player

   sessions.csv variables
	•	hashedEmail — character ID linking sessions to players
	•	start_time — datetime string; session start
	•	end_time — datetime string; session end
	•	original_start_time — numeric; rounded Unix timestamp
	•	original_end_time — numeric; rounded Unix timestamp
- created variables:
    -   duration — numeric; minutes per session
	•	total_session_minutes — numeric; total minutes per player
	•	mean_session_minutes — numeric; average session length
	•	number_of_sessions — integer; count of sessions per player
- Issues Observed in the Data
	•	original_start_time and original_end_time are too rounded, causing inaccurate durations
	•	Some session durations are extremely long (possible AFK / idle time)
	•	Certain experience levels have very small counts (imbalanced categories)
- Potential Hidden Issues (Things Not Directly Observable)
	•	Sampling bias (players who join a study server ≠ general population)
	•	Possible time zone discrepancies in timestamps
	•	Missing or incomplete session logs for some players
- Data collected via plaicraft.ai program launched by The Pacific Laboratory for Artificial Intelligence with volunteers
  


2.Questions
Broad question:
Which kinds of players contribute the most gameplay data?

Specific question:
Can player characteristics (experience level, age, gender, total played hours) predict whether a player is a high or low data contributor based on each player’s total time spent in all recorded sessions?

To define the response variable, I will summarize the sessions data for each player:
	•	First compute the duration of every session in sessions.csv using
session_minutes = end_time − start_time
	•	Then aggregate total session time per player:
total_session_minutes
	•	To classify players, I will use the median total session time as a threshold:
	•	HighData = 1 if above median
	•	LowData = 0 otherwise

This creates a balanced binary classification target appropriate for KNN.

3.Exploratory Data Analysis and Visualization

In [ ]:
average_gender <- combined_data |>
  group_by(gender) |>
  summarize(count = n()) |>
  mutate(percentage = round(count / sum(count) * 100, 2))
average_gender

In [ ]:
average_experience <- combined_data |>
  group_by(experience) |>
  summarize(count = n()) |>
  mutate(percentage = round(count / sum(count) * 100, 2)) |>
  arrange(desc(count))
average_experience

In [ ]:
average_across <- combined_data |>
  summarize(across(where(is.numeric), ~round(mean(.x, na.rm = TRUE), 2)))
average_across 

In [ ]:
sessions <- sessions |>
  mutate(original_end_time = as.numeric(as.POSIXct(end_time, format = "%d/%m/%Y %H:%M")),
         original_start_time = as.numeric(as.POSIXct(start_time, format = "%d/%m/%Y %H:%M")))
sessions

In [ ]:
session_duration <- sessions |>
    mutate(duration = original_end_time - original_start_time) |>
    select(duration)
sessions_summary <- session_duration |>
    summarize(min_duration = min(duration, na.rm = TRUE),
              Q1_duration = quantile(duration, 0.25, na.rm = TRUE),
              median_duration = median(duration, na.rm = TRUE),
              Q2_duration = quantile(duration, 0.75, na.rm = TRUE),
              max_duration = max(duration, na.rm = TRUE),
              mean_duration = mean(duration, na.rm = TRUE))
sessions_summary

|Variable|Value|
|-------|------|
| Play hours | 98.57|
| Age        | 19.43|

In [ ]:
age_distribution <- combined_data |>
  ggplot(aes(x = Age)) +
  geom_histogram(bins = 30) +
  xlab("Age(years)") +
  ylab("Number of Players") +
  ggtitle("Age Distribution of Players in Years") +
  theme(text = element_text(size = 20))
age_distribution

In [ ]:
Played_Hours <- players |>
    ggplot(aes(x = played_hours)) +
    geom_histogram(binwidth = 10) +
    ggtitle("Distribution of Player Total Playtime") +
    xlab("Total Hours Played") +
    ylab("Number of Players")
Played_Hours

4. Methods and Plans
   To address my question of whether player characteristics can predict whether a player is a high-data or low-data contributor, I propose using a K-Nearest Neighbours (KNN) classification model. 

Assumptions. KNN requires that distances meaningfully represent similarity, so all numeric variables (Age, played_hours, session-based variables) must be standardized. Categorical variables such as gender and experience will need to be converted into dummy variables. The method also assumes that nearby points in feature space tend to share the same class.

Limitations. KNN is sensitive to irrelevant predictors, noise, and differences in scale; it may also perform poorly if classes are imbalanced or if there are many dummy variables, which increases dimensionality. KNN may become less reliable in sparse regions of the data.

Model comparison and selection. I will evaluate different values of k using classification error on a validation set or through cross-validation within the training set. The value of k that minimizes error will be chosen as the final model.

Data processing plan. Before modeling, I will: (1) merge the datasets and compute total_session_minutes; (2) convert categorical variables into dummy variables; (3) standardize numeric predictors; (4) split the data into training and testing sets using an 80/20 split. The split will occur after preprocessing but before model fitting. Cross-validation may be used within the training set to choose the best k.

This approach provides a clear and justified method for predicting which kinds of players contribute the most gameplay data.